In diesem Projekt greifen wir auf ein neues Konzept zurück, um die Qualität und Effizienz von Lösungen zu verbessern. Wir nutzen zwei Sprachmodelle, einen Generator und einen Critic, um eine Feedback-Schleife zu bilden. Diese Methode basiert auf der sogenannten Self-Consistency mit Feedback-Schleife oder Critic-and-Generator-Framework, die es ermöglicht, sowohl die Qualität der Lösung als auch ihre Erklärbarkeit zu erhöhen.


Der Generator-Modell erstellt eine Lösung und gibt dabei durch Chain-of-Thought Prompting eine schrittweise Erklärung ab. Der Critic-Modell bewertet den Vorschlag des Generators und identifiziert mögliche Schwächen oder Fehler. Dieser Prozess wiederholt sich in einer Feedback-Schleife, wodurch der Generator seine Lösung kontinuierlich verbessern kann.


Diese innovative Methode bietet mehrere Vorteile: Sie ermöglicht eine Qualitätskontrolle durch den Critic, schrittweise Optimierung des Generators und erhöhte Erklärbarkeit durch Chain-of-Thought-Prompting. Darüber hinaus ist sie flexibel und anpassbar für verschiedene Aufgaben und Anwendungsfälle.


In den folgenden Abschnitten wird aufgezeigt, wie diese Methode umgesetzt wird und welche Vorteile sie bietet. Wir werden uns auf die Implementierung des Generators und Critic-Modells konzentrieren, sowie auf die Ergebnisse der Feedback-Schleife und die daraus gewonnenen Erkenntnisse.

### Aufgabe

Zeichne den Punkt P(4|6) in ein Koordinatensystem ein.

### Soll Lösung

1. Zeichne ein Koordinatensystem mit einer x-Achse und einer y-Achse, damit beide der Punkt im Koordinatensystem liegt <br>
		Achte darauf, die Achsen zu beschriften und eine regelmäßige Skalierung zu verwenden. <br>
2. Markiere den Ursprung (0, 0) des Koordinatensystems deutlich.<br>
3. Zeichne einen Vektor in x-Richtung, der von (0, 0) bis zum X Punkt reicht (x,  0). <br>
		Nutze eine klare Farbe wie Rot, um diesen Vektor zu kennzeichnen. <br>
4. Zeichne einen Vektor in y-Richtung, der von (0, 0) bis zum Y Punkt reicht (0, y). <br>
		Verwende eine andere Farbe, z. B. Grün. <br>
5. Verschiebe den y-Richtungs-Vektor so, dass sein Startpunkt das Ende des x-Richtungs-Vektors bei (x, 0) ist. <br>
6. Der Endpunkt dieses verschobenen Vektors markiert die Koordinaten (x, y).  <br>
		Zeichne einen Punkt an dieser Stelle und beschrifte ihn mit 'P({x-Wert}|{y-Wert})'. <br>
7. Füge ggf. Beschriftungen oder zusätzliche Hinweise hinzu, um die Schritte der Konstruktion verständlicher zu machen. <br>


In [14]:
from openai import OpenAI

# Verbindung zum Ollama-Server herstellen
client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama',  # erforderlich, aber ungenutzt
)

# Funktion zum Generieren der Schritt-für-Schritt-Anleitung
def generate_visual_steps(task_description):
    response = client.chat.completions.create(
        model="llama3.2-vision",  # Generator-Modell
        messages=[
            {"role": "system", "content": "Du bist ein Sprachmodell, das Mathematikaufgaben in visuell darstellbare Schritte zerlegt."},
            {"role": "user", "content": task_description},
        ]
    )
    # Zugriff auf die generierte Antwort
    return response.choices[0].message.content  # Korrigierter Zugriff auf 'content'

# Funktion zur Bewertung des Ergebnisses
def evaluate_solution(task_description, generated_solution):
    response = client.chat.completions.create(
        model="llama3.2-vision",  # Bewertungsmodell
        messages=[
            {"role": "system", "content": "Du bist ein Sprachmodell, das Lösungen für Mathematikaufgaben bewertet. Gib eine Antwort, ob die Lösung klar und korrekt ist."},
            {"role": "user", "content": f"Aufgabe:\n{task_description}\n\nLösung:\n{generated_solution}"},
        ]
    )
    # Zugriff auf die Bewertungsantwort
    return response.choices[0].message.content  # Korrigierter Zugriff auf 'content'

# Funktion zur Steuerung der Interaktion zwischen Generator und Evaluator
def interactive_solution(task_description, max_iterations=2):
    iteration = 0
    solution = generate_visual_steps(task_description)
    print(f"Iteration {iteration + 1} - Generierte Lösung:\n{solution}")
    
    while iteration < max_iterations:
        # Bewertungsmodell aufrufen
        evaluation = evaluate_solution(task_description, solution)
        print(f"Iteration {iteration + 1} - Bewertung der Lösung:\n{evaluation}")
        
        if "korrekt" in evaluation.lower() or "klar" in evaluation.lower():
            print("Lösung akzeptiert!")
            break  # Wenn die Lösung akzeptiert wird, Schleife beenden
        else:
            # Falls Bewertung negativ, neue Lösung generieren
            iteration += 1
            if iteration < max_iterations:
                print(f"Neue Iteration {iteration + 1}: Verbesserung der Lösung...")
                solution = generate_visual_steps(task_description)
                print(f"Iteration {iteration + 1} - Neue generierte Lösung:\n{solution}")
            else:
                print("Maximale Anzahl an Iterationen erreicht. Keine weiteren Verbesserungen möglich.")

# Beispielaufgabe
task_description = "Zeichne den Punkt P(4|6) in ein Koordinatensystem ein."

# Ablauf mit maximal 2 Iterationen starten
interactive_solution(task_description)

Iteration 1 - Generierte Lösung:
Um die Punkte zu zeichnen, folge bitte diesen Anweisungen:

1. Zuerst machen wir 4 Schritte nach rechts und dann 6 Schritte nach oben.

2. Wir können den Punkt als Kreuz markieren.

Ich kann dir eine einfache Grafik zeichnen, aber leider fehlt mir die Möglichkeit, Bilder in diesem Format zu erstellen.
Iteration 1 - Bewertung der Lösung:
Die Lösung ist klar und korrekt!

Sie beschreibt genau, wie man einen Punkt mit der Koordinate (4|6) in ein Koordinatensystem einzeichnen kann:

1. Mach 4 Schritte nach rechts von der x-Achse.
2. Dann mach 6 Schritte nach oben von der y-Achse.

Durch diese Anweisungen entsteht exakt die Punktekoordinate (4,6).

Die ergänzende Bemerkung zur Kreuzmarkierung des Punkts ist überflüssig für dieses Problem, aber sie hilft dabei, den Punkt konsistent und leichter erkennbar darzustellen.

Die Lösung kann somit als akkurat betrachtet werden.
Lösung akzeptiert!


In [29]:
def generate_visual_steps(task_description):
    response = client.chat.completions.create(
        model="phi4",  # Generator-Modell
        messages=[
            {"role": "system", "content": (
                "Du bist ein Sprachmodell, das Mathematikaufgaben in präzise, nummerierte Schritte zerlegt, "
                "die leicht visuell dargestellt werden können. Gib die Antwort als eine Schritt-für-Schritt-Liste zurück, "
                "beginnend mit '1.', '2.', usw. Jede Nummer sollte einen klar definierten Schritt enthalten."
            )},
            {"role": "user", "content": task_description},
        ]
    )
    return response.choices[0].message.content  # Zugriff auf 'content'

In [30]:
def evaluate_solution(task_description, generated_solution):
    response = client.chat.completions.create(
        model="phi4",  # Bewertungsmodell
        messages=[
            {"role": "system", "content": (
                "Du bist ein Sprachmodell, das Lösungen für Mathematikaufgaben bewertet. "
                "Bewerte die Lösung nicht nur auf Korrektheit, sondern auch darauf, ob sie die folgenden Kriterien erfüllt:\n"
                "1. Die Lösung muss als nummerierte Schritt-für-Schritt-Anleitung vorliegen.\n"
                "2. Die Schritte müssen klar und präzise formuliert sein.\n"
                "3. Begriffe wie 'Koordinatensystem', 'x-Achse', 'y-Achse', 'Vektor' sollten verwendet werden, "
                "wenn sie für die Aufgabe relevant sind.\n"
                "Antworte mit einer kurzen Bewertung und sage klar, ob die Lösung akzeptiert wird oder nicht."
            )},
            {"role": "user", "content": f"Aufgabe:\n{task_description}\n\nLösung:\n{generated_solution}"},
        ]
    )
    return response.choices[0].message.content  # Zugriff auf 'content'

In [31]:
def interactive_solution(task_description, max_iterations=2):
    iteration = 0
    solution = generate_visual_steps(task_description)
    final_solution = None  
    print(f"Iteration {iteration + 1} - Generierte Lösung:\n{solution}")
    
    while iteration < max_iterations:
        evaluation = evaluate_solution(task_description, solution)
        print(f"Iteration {iteration + 1} - Bewertung der Lösung:\n{evaluation}")
        
        if "akzeptiert" in evaluation.lower() and "nummeriert" in evaluation.lower():
            print("Lösung akzeptiert!")
            final_solution = solution
            break
        else:
            iteration += 1
            if iteration < max_iterations:
                print(f"Neue Iteration {iteration + 1}: Verbesserung der Lösung...")
                solution = generate_visual_steps(task_description)
                print(f"Iteration {iteration + 1} - Neue generierte Lösung:\n{solution}")
            else:
                print("Maximale Anzahl an Iterationen erreicht. Keine weiteren Verbesserungen möglich.")
    
    if final_solution is None:
        final_solution = solution
    return final_solution  

In [32]:
# Beispielaufgabe
task_description = "Zeichne den Punkt P(4|6) in ein Koordinatensystem ein."

# Ablauf mit maximal 2 Iterationen starten
final_solution = interactive_solution(task_description)

Iteration 1 - Generierte Lösung:
Um den Punkt \( P(4, 6) \) in einem kartesischen Koordinatensystem einzutragen, folge diesen Schritten:

1. **Koordinatenachsen Zeichnen**:
   - Mal eine horizontale Linie für die x-Achse (Abszisse).
   - Mal eine vertikale Linie für die y-Achse (Ordinate).
   - Kreuzen Sie die beiden Achsen, wobei der Schnittpunkt \( (0, 0) \) ist, genannt Ursprung.

2. **Ausschildern**:
   - Beauftrage die positive Richtung der x-Achse nach rechts und markiere mit positiven ganzen Zahlen: 1, 2, 3, usw.
   - Beauftrage die negative Richtung der x-Achse nach links und markiere mit negativen ganzen Zahlen: -1, -2, -3, usw.

3. **Y-Achsengestaltung**:
   - In die positive Richtung der y-Achse gehe oben und benenne diese mit positiven ganzen Zahlen: 1, 2, 3, usw.
   - In unten gelegene negative Richtung der y-Achse benenne sie im Gegenzug mit negativen ganzen Zahlen: -1, -2, -3, usw.

4. **Den Punkt Finden**:
   - Nimm den x-Wert des Punktes \( P \), welcher 4 ist.
   - Vo

In [33]:
print("Endgültige Lösung:")
print(final_solution)

Endgültige Lösung:
Um den Punkt \( P(4, 6) \) in einem kartesischen Koordinatensystem einzutragen, folge diesen Schritten:

1. **Koordinatenachsen Zeichnen**:
   - Mal eine horizontale Linie für die x-Achse (Abszisse).
   - Mal eine vertikale Linie für die y-Achse (Ordinate).
   - Kreuzen Sie die beiden Achsen, wobei der Schnittpunkt \( (0, 0) \) ist, genannt Ursprung.

2. **Ausschildern**:
   - Beauftrage die positive Richtung der x-Achse nach rechts und markiere mit positiven ganzen Zahlen: 1, 2, 3, usw.
   - Beauftrage die negative Richtung der x-Achse nach links und markiere mit negativen ganzen Zahlen: -1, -2, -3, usw.

3. **Y-Achsengestaltung**:
   - In die positive Richtung der y-Achse gehe oben und benenne diese mit positiven ganzen Zahlen: 1, 2, 3, usw.
   - In unten gelegene negative Richtung der y-Achse benenne sie im Gegenzug mit negativen ganzen Zahlen: -1, -2, -3, usw.

4. **Den Punkt Finden**:
   - Nimm den x-Wert des Punktes \( P \), welcher 4 ist.
   - Von dem Ursprung

# Beispiel Aufgaben
---

In [34]:
# Beispielaufgabe
task_description = "Zeichne einen Kreis mit Mittelpunkt M(0|0) und Radius 5 in ein Koordinatensystem."

# Ablauf mit maximal 2 Iterationen starten
final_solution = interactive_solution(task_description)

Iteration 1 - Generierte Lösung:
Um einen Kreis mit dem gegebenen Mittelpunkt \( M(0, 0) \) und einem Radius von 5 zu zeichnen, folge diesen Schritten:

1. **Koordinatensystem aufrichten**: Zeichne ein horizontales X-Achse- und ein vertikales Y-Achsen-Liniennetz auf ein Blatt Papier oder im graphischen Programm. Stelle sicher, dass beide Achsen ordnungsgemäß beschriftet und gekennzeichnet sind (wobei das Zentrum der Achsen den Punkt \( M(0, 0) \) darstellt).

2. **Mittelpunkt identifizieren**: Markiere den Mittelpunkt des Kreises an der Schnittposition der X- und Y-Achse - was der Ursprung ist, gekennzeichnet als \( M(0, 0) \).

3. **Radius bestimmen**: Der gegebene Radius beträgt 5 Einheiten.

4. **Nördlicher Punkt aufzeichnen**: Vom Mittelpunkt aus zeichne eine vertikale Linie gerade nach oben (positiver y-Richtung) für 5 Einheiten und markiere den Endpunkt dieser Linie bei \( (0, 5) \).

5. **Südlicher Punkt aufzeichnen**: Zeichne eine vertikale Linie vom Mittelpunkt aus nach unten 

In [35]:
print("Endgültige Lösung:")
print(final_solution)

Endgültige Lösung:
Um einen Kreis mit dem gegebenen Mittelpunkt \( M(0, 0) \) und einem Radius von 5 zu zeichnen, folge diesen Schritten:

1. **Koordinatensystem aufrichten**: Zeichne ein horizontales X-Achse- und ein vertikales Y-Achsen-Liniennetz auf ein Blatt Papier oder im graphischen Programm. Stelle sicher, dass beide Achsen ordnungsgemäß beschriftet und gekennzeichnet sind (wobei das Zentrum der Achsen den Punkt \( M(0, 0) \) darstellt).

2. **Mittelpunkt identifizieren**: Markiere den Mittelpunkt des Kreises an der Schnittposition der X- und Y-Achse - was der Ursprung ist, gekennzeichnet als \( M(0, 0) \).

3. **Radius bestimmen**: Der gegebene Radius beträgt 5 Einheiten.

4. **Nördlicher Punkt aufzeichnen**: Vom Mittelpunkt aus zeichne eine vertikale Linie gerade nach oben (positiver y-Richtung) für 5 Einheiten und markiere den Endpunkt dieser Linie bei \( (0, 5) \).

5. **Südlicher Punkt aufzeichnen**: Zeichne eine vertikale Linie vom Mittelpunkt aus nach unten (negativer y-R

---

In [36]:
# Beispielaufgabe
task_description = "Zeichne die Gerade durch die Punkte A(1|2) und B(4|5) in ein Koordinatensystem."

# Ablauf mit maximal 2 Iterationen starten
final_solution = interactive_solution(task_description)

Iteration 1 - Generierte Lösung:
Um die Gerade durch die Punkte \(A(1, 2)\) und \(B(4, 5)\) zu zeichnen, gehen wir wie folgt vor:

1. **Bestimme den Steigungswert (m):**
   - Die Formel für den Steigungswert zwischen zwei Punkten \((x_1, y_1)\) und \((x_2, y_2)\) ist:
     \[
     m = \frac{y_2 - y_1}{x_2 - x_1}
     \]
   - Setze die Koordinaten der Punkte ein: \(A(1, 2)\) und \(B(4, 5)\):
     \[
     m = \frac{5 - 2}{4 - 1} = \frac{3}{3} = 1
     \]

2. **Finde die Gleichung der Geraden in Punkt-Steigungsform:**
   - Die Punkt-Steigungs-Gleichung ist:
     \[
     y - y_1 = m(x - x_1)
     \]
   - Wähle den Punkt \(A(1, 2)\) und setze \(m\) ein:
     \[
     y - 2 = 1(x - 1)
     \]

3. **Vereinfache die Gleichung:**
   - Vereinfache die obige Gleichung:
     \[
     y - 2 = x - 1
     \]
   - Führe \(y\)-Isolierung durch, um in der y-Achsenabschnittsform zu schreiben:
     \[
     y = x + 1
     \]

4. **Zeichne die Punkte:**
   - Beginne mit dem Zeichnen eines Koordinatennetzes au

In [37]:
print("Endgültige Lösung:")
print(final_solution)

Endgültige Lösung:
Um die Gerade durch die Punkte \(A(1, 2)\) und \(B(4, 5)\) zu zeichnen, gehen wir wie folgt vor:

1. **Bestimme den Steigungswert (m):**
   - Die Formel für den Steigungswert zwischen zwei Punkten \((x_1, y_1)\) und \((x_2, y_2)\) ist:
     \[
     m = \frac{y_2 - y_1}{x_2 - x_1}
     \]
   - Setze die Koordinaten der Punkte ein: \(A(1, 2)\) und \(B(4, 5)\):
     \[
     m = \frac{5 - 2}{4 - 1} = \frac{3}{3} = 1
     \]

2. **Finde die Gleichung der Geraden in Punkt-Steigungsform:**
   - Die Punkt-Steigungs-Gleichung ist:
     \[
     y - y_1 = m(x - x_1)
     \]
   - Wähle den Punkt \(A(1, 2)\) und setze \(m\) ein:
     \[
     y - 2 = 1(x - 1)
     \]

3. **Vereinfache die Gleichung:**
   - Vereinfache die obige Gleichung:
     \[
     y - 2 = x - 1
     \]
   - Führe \(y\)-Isolierung durch, um in der y-Achsenabschnittsform zu schreiben:
     \[
     y = x + 1
     \]

4. **Zeichne die Punkte:**
   - Beginne mit dem Zeichnen eines Koordinatennetzes auf Papier oder 

---

In [38]:
# Beispielaufgabe
task_description = "Zeichne das Dreieck mit den Eckpunkten A(1|1), B(5|1), und C(3|4) in ein Koordinatensystem. Beschrifte die Punkte und verbinde sie."

# Ablauf mit maximal 2 Iterationen starten
final_solution = interactive_solution(task_description)

Iteration 1 - Generierte Lösung:
Um das Dreieck mit den gegebenen Eckpunkten \(A(1, 1)\), \(B(5, 1)\) und \(C(3, 4)\) zu zeichnen, befolge diese Schritte:

1. **Raster erstellen**: Zeichne ein Koordinatensystem auf Papier oder einem Bildschirm. Markiere die x- und y-Achsen zur Orientierung. Das Raster sollte groß genug sein, um alle gegebene Punkte darzustellen.

2. **Punkte markieren**:
   - **Punkt A**: Lokalisiere \((1, 1)\) auf dem Koordinatensystem. Bewege dich 1 Einheit entlang der x-Achse von der Ursprungspunkt nach rechts und dann 1 Einheit entlang der y-Achse hoch. Markiere diesen Punkt als A.
   - **Punkt B**: Finde \((5, 1)\) auf dem Koordinatensystem. Bewege dich 5 Einheiten entlang der x-Achse von der Ursprungspunkt nach rechts und bleibe dann auf derselben Höhe (y=1). Markiere diesen Punkt als B.
   - **Punkt C**: Finde \((3, 4)\) auf dem Koordinatensystem. Bewege dich 3 Einheiten entlang der x-Achse von der Ursprungspunkt nach rechts und dann 4 Einheiten entlang der y-Ac

In [39]:
print("Endgültige Lösung:")
print(final_solution)

Endgültige Lösung:
Um das Dreieck mit den gegebenen Eckpunkten \(A(1, 1)\), \(B(5, 1)\) und \(C(3, 4)\) zu zeichnen, befolge diese Schritte:

1. **Raster erstellen**: Zeichne ein Koordinatensystem auf Papier oder einem Bildschirm. Markiere die x- und y-Achsen zur Orientierung. Das Raster sollte groß genug sein, um alle gegebene Punkte darzustellen.

2. **Punkte markieren**:
   - **Punkt A**: Lokalisiere \((1, 1)\) auf dem Koordinatensystem. Bewege dich 1 Einheit entlang der x-Achse von der Ursprungspunkt nach rechts und dann 1 Einheit entlang der y-Achse hoch. Markiere diesen Punkt als A.
   - **Punkt B**: Finde \((5, 1)\) auf dem Koordinatensystem. Bewege dich 5 Einheiten entlang der x-Achse von der Ursprungspunkt nach rechts und bleibe dann auf derselben Höhe (y=1). Markiere diesen Punkt als B.
   - **Punkt C**: Finde \((3, 4)\) auf dem Koordinatensystem. Bewege dich 3 Einheiten entlang der x-Achse von der Ursprungspunkt nach rechts und dann 4 Einheiten entlang der y-Achse hoch. Mark

---